In [9]:
import random
import json
import pickle
import numpy as np

import nltk # Natural language Toolkit
from nltk.stem import WordNetLemmatizer
#Does not loose performance when we have exact words
#For example: work, worked, working are treated as same

import tensorflow
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.optimizers import Adam

#Create a lemmatizer
lemmatizer = WordNetLemmatizer()

#Open json file
intents = json.loads(open('intents.json').read())

#Creating Lists for words ,classes,documents
words=[]
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
    '''Iterate through intents dictionary'''
    for pattern in intent['patterns']:
        '''Iterate through items in pattern'''
        word_list = nltk.word_tokenize(pattern)
        #The creates tokens of words and stores in the list
        #Add the collection of words to the word list
        words.extend(word_list)
        documents.append(((word_list),intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words= [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

#We create a bag of words to assign numerical values
#0 and 1 to distinguish the occurence of the word in the pattern
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        '''Check if that word occurs in pattern'''
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])]
    training.append([bag,output_row])


random.shuffle(training)
training = np.array(training,dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])

model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y),activation='softmax'))

sgd = Adam(learning_rate=0.01,epsilon=1e-07)
model.compile(loss='categorical_entropy',optimizer=sgd,metrics=['accuracy'])

hist = model.fit(np.array(train_x),np.array(train_y),epoch=200,batch_size=5,verbose=1)

model.save('chatbot_model.h5',hist)
print("Done")




ValueError: Could not interpret optimizer identifier: <keras.optimizers.optimizer_experimental.adam.Adam object at 0x000001D38BCFC2B0>